# Проверка гипотез

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as sts

In [2]:
cars_df = pd.read_csv('cars_df_final.csv')

In [3]:
cars_df['Accidents or damage'] = (cars_df['Accidents or damage'] == 'At least 1 accident or damage reported')
cars_df['1-owner vehicle'] = (cars_df['1-owner vehicle'] == 'Yes')
cars_df['Personal use only'] = (cars_df['Personal use only'] == 'Yes')

In [4]:
print(cars_df.shape)
cars_df.head()

(2542, 23)


,Brand,Year,Exterior color,Interior color,Drivetrain,Fuel type,Capacity,Configuration,Valves,Accidents or damage,...,MPG_max,MPG_mean,Mileage,Usage Intensity,Ownership History,Engine Power,Fuel Efficiency,Origin,Gallons Per Year,Price
0,Hyundai,2022,Red,Black,Front-wheel Drive,Gasoline,2.5,I4,16.0,True,...,28.000000,25.000000,13256.0,6628.000,Potential Risk,40.0,10.000000,Asian,265.120000,38988.0
1,Lexus,2016,Black,Black,All-wheel Drive,Gasoline,3.5,V6,24.0,True,...,26.000000,22.500000,100067.0,12508.375,Bad,84.0,6.428571,Asian,555.927778,19747.0
2,Chevrolet,2017,White,Black,Front-wheel Drive,Hybrid,1.5,I4,16.0,True,...,27.239548,23.916163,39032.0,5576.000,Potential Risk,24.0,15.944109,American,233.147769,25999.0
3,Mercedes-Benz,2022,White,Other,All-wheel Drive,Gasoline,3.0,I6,24.0,False,...,27.239548,23.916163,10901.0,5450.500,Good,72.0,7.972054,European,227.900272,96750.0
4,Land Rover,2020,Blue,Other,Four-wheel Drive,Gasoline,3.0,I6,24.0,False,...,22.000000,19.500000,24946.0,6236.500,Good,72.0,6.500000,British,319.820513,61530.0


In [5]:
# Получим списки автомобилей по происхождению
american_cars = ['Chevrolet', 'Chrysler', 'GMC', 'Cadillac',
                 'Ford', 'Buick', 'Jeep', 'Lincoln', 'RAM',
                 'Pontiac', 'Dodge', 'Hummer', 'Plymouth']

european_cars = ['Mercedes-Benz', 'Audi', 'BMW', 'Lamborghini', 'Porsche', 'Volkswagen',
                 'Volvo', 'FIAT', 'MINI', 'Maserati', 'Alfa Romeo', 'Ferrari', 'Saab']

british_cars = ['Land Rover', 'McLaren', 'Jaguar', 'Aston Martin', 'Bentley', 'Lotus', 'Rolls-Royce']

asian_cars = ['Hyundai', 'Lexus', 'INFINITI', 'Mazda', 'Toyota', 'Genesis', 'Kia',
              'Acura', 'Nissan', 'Honda', 'Mitsubishi', 'Scion', 'Suzuki']

# Гипотезы

__Заметка:__ Поверим, что цены автомобилей независимы. Также заметим, что у нас в наличии большое число наблюдений, следовательно, мы можем пользоваться тем, что среднее значение какого-либо признака будет распределено асимптотически нормально. Наконец, установим уровень значимости $0.05$ для каждой проверки.

__1)__ Не менее 10% автомобилей, размещенных на cars.com, участвовали в ДТП.

В данном случае мы имеем дело с долями. Пусть $p$ - вероятность того, что автомобиль попал в ДТП.

Тогда $H_{0}: p = 0.1,$ $H_{1}: p < 0.1 \overset{H_{0}}{\Rightarrow} z_{obs} = \frac {\hat {p} - p} {\sqrt {\frac {\hat {p} (1 - \hat {p})} {n}}} \overset{asy}{\sim} N(0, 1), \space z_{crit} = -z_{0.95}.$


In [6]:
n = cars_df.shape[0]
p_hat = cars_df['Accidents or damage'].mean()

z_obs = (p_hat - 0.1) / (np.sqrt(p_hat * (1 - p_hat) / n))
z_crit = -sts.norm.ppf(0.95)
print(z_obs, z_crit)

# Видим, что наблюдаемое значение больше критического
z_obs > z_crit

12.668971915279856 -1.6448536269514722


True

__Вывод:__ Гипотеза о том, не менее 10% автомобилей, размещенных на cars.com, участвовали в ДТП, **не** отвергается на данном уровне значимости.

---

__2)__ Автомобили, поучаствовавшие в ДТП, в среднем стоят дешевле на 20% и более.

Здесь рассмотрим две выборки, независимые по условию: $X$ - цена автомобилей, участвовавших в ДТП, $Y$ - не участвовавших.

Тогда $H_{0}: \mu_{x} = 0.8 \mu_{Y}, \space H_{1}: \mu_{X} > 0.8 \mu_{Y} \Rightarrow$ рассмотрим случайную величину $\bar{X} - 0.8 \bar{Y} \overset{asy}{\sim} N(\mu_{Y} - 0.8 \mu_{Y}, \frac {\sigma_{X}^2} {n_{X}} + 0.64 \frac {\sigma_{Y}^2} {n_{Y}}) \overset{H_{0}}{\Rightarrow}$

$\overset{H_{0}}{\Rightarrow} z_{obs} = \frac {\bar{X} - \scriptstyle 0.8 \bar{Y}} {\sqrt{\frac {\hat{\sigma}_{X}^2} {n_{X}} + \scriptstyle 0.64 \frac {\hat{\sigma}_{Y}^2} {n_{Y}}}} \overset{asy}{\sim} N(0, 1), \space z_{crit} = z_{0.95}.$

In [7]:
X_cond = cars_df['Accidents or damage']
X = cars_df[X_cond]['Price'].values

Y = cars_df[~X_cond]['Price'].values

In [8]:
n_X = X.shape[0]
n_Y = Y.shape[0]

X_mean = X.mean()
Y_mean = Y.mean()

X_var = X.var(ddof=1)
Y_var = Y.var(ddof=1)

In [9]:
z_obs = (X_mean - 0.8 * Y_mean) / np.sqrt(X_var / n_X + 0.64 * Y_var / n_Y)
z_crit = sts.norm.ppf(0.95)
print(z_obs, z_crit)

# Видим, что наблюдаемое значение меньше критического
z_obs < z_crit

-3.9784484748131557 1.6448536269514722


True

__Вывод:__ Гипотеза о том, что автомобили, поучаствовавшие в ДТП, в среднем стоят дешевле на 20% и более, **не** отвергается на данном уровне значимости. Получается, что вполне очевидное заключение о том, что ДТП снижает цену, подтверждается и статистически.

---

__3)__ Американские и европейские автомобили в среднем стоят одинаково.

Аналогично рассмотрим две незавизимые выборки: $X$ - цена американских автомобилей, $Y$ - цена европейских автомибилей.

Тогда $H_{0}: \mu_{X} = \mu_{Y}, \space H_{1}: \mu_{X} \neq \mu_{Y} \overset{H_{0}}{\Rightarrow} z_{obs} = \frac {\bar{X} - \bar{Y}} {\sqrt{\frac {\hat{\sigma}_{X}^2} {n_{X}} + \frac {\hat{\sigma}_{Y}^2} {n_{Y}}}} \overset{asy}{\sim} N(0, 1), \space z_{crit} = z_{0.975}.$

In [10]:
X_cond = np.vectorize(lambda x: x in american_cars)(cars_df['Brand'])
X = cars_df[X_cond]['Price'].values

Y_cond = np.vectorize(lambda x: x in european_cars)(cars_df['Brand'])
Y = cars_df[Y_cond]['Price'].values

In [11]:
n_X = X.shape[0]
n_Y = Y.shape[0]

X_mean = X.mean()
Y_mean = Y.mean()

X_var = X.var(ddof=1)
Y_var = Y.var(ddof=1)

In [12]:
z_obs = (X_mean - Y_mean) / np.sqrt(X_var / n_X + Y_var / n_Y)
z_crit = sts.norm.ppf(0.975)
print(z_obs, z_crit)

# Видим, что наблюдаемое значение по модулю превышает критическое.
np.abs(z_obs) > z_crit

-3.0812372919188302 1.959963984540054


True

__Вывод:__ Гипотеза о том, что американские и европейские автомобили в среднем стоят одинаково, отвергается на данном уровне значимости.

---

__4)__ 90% и более автомобилей ездят на бензине.

Пусть $p$ - вероятность того, что автомобиль ездит на бензине.

Тогда $H_{0}: p = 0.9, \space H_{1}: p < 0.9 \overset{H_{0}}{\Rightarrow} z_{obs} = \frac {\hat{p} - 0.9} {\sqrt{\frac {\hat{p} (1 - \hat{p})} {n}}} \overset{asy}{\sim} N(0, 1), \space z_{crit} = -z_{0.95}$

In [13]:
n = cars_df.shape[0]
p_hat = (cars_df['Fuel type'] == 'Gasoline').mean()

z_obs = (p_hat - 0.9) / (np.sqrt(p_hat * (1 - p_hat) / n))
z_crit = -sts.norm.ppf(0.95)
print(z_obs, z_crit)

# Видим, что наблюдаемое значение меньше критического
z_obs < z_crit

-3.9342925480047013 -1.6448536269514722


True

__Вывод:__ Гипотеза о том, что 90% и более автомобилей ездят на бензине, отвергается на данном уровне значимости.

---

__5)__ Большинство автомобилей имеет передний привод.

Пусть $p$ - вероятность того, что автомобиль имеет передний привод.

Тогда $H_{0}: p = 0.5, \space H_{1}: p < 0.5 \overset{H_{0}}{\Rightarrow} z_{obs} = \frac {\hat{p} - 0.5} {\sqrt{\frac {\hat{p} (1 - \hat{p})} {n}}} \overset{asy}{\sim} N(0, 1), \space z_{crit} = -z_{0.95}$

In [14]:
n = cars_df.shape[0]
p_hat = (cars_df['Drivetrain'] == 'Front-wheel Drive').mean()

z_obs = (p_hat - 0.5) / (np.sqrt(p_hat * (1 - p_hat) / n))
z_crit = -sts.norm.ppf(0.95)
print(z_obs, z_crit)

# Видим, что наблюдаемое значение меньше критического
z_obs < z_crit

-30.31869999086294 -1.6448536269514722


True

__Вывод:__ Гипотеза о том, что большинство автомобилей имеет передний привод, отвергается на данном уровне значимости.

---

__6)__ Самые современные автомобили (новее 2020 года включительно) имеют сравнимый или больший средний объем двигателя, чем автомобили старее.

Все также рассмотрим две независимые выборки: $X$ - объем двигателя автомобилей новее 2020 года включительно, $Y$ - объем двигателя автомобилей старее.

Тогда $H_{0}: \mu_{X} = \mu_{Y}, \space H_{1}: \mu_{X} < \mu_{Y} \overset{H_{0}}{\Rightarrow} z_{obs} = \frac {\bar{X} - \bar{Y}} {\sqrt{\frac {\hat{\sigma}_{X}^2} {n_{X}} + \frac {\hat{\sigma}_{Y}^2} {n_{Y}}}} \overset{asy}{\sim} N(0, 1), \space z_{crit} = z_{0.95}.$

In [15]:
X_cond = (cars_df['Year'] >= 2020)
X = cars_df[X_cond]['Capacity'].values

Y = cars_df[~X_cond]['Capacity'].values

In [16]:
n_X = X.shape[0]
n_Y = Y.shape[0]

X_mean = X.mean()
Y_mean = Y.mean()

X_var = X.var(ddof=1)
Y_var = Y.var(ddof=1)

In [17]:
z_obs = (X_mean - Y_mean) / np.sqrt(X_var / n_X + Y_var / n_Y)
z_crit = -sts.norm.ppf(0.95)
print(z_obs, z_crit)

# Видим, что наблюдаемое значение меньше критического
z_obs < z_crit

-6.572510541601721 -1.6448536269514722


True

__Вывод:__ Гипотеза о том, что самые современные автомобили имеют сравнимый или больший средний объем двигателя, чем автомобили старее, отвергается на данном уровне значимости.

---

__7)__ Американские автомобили в среднем расходуют на 10% больше топлива, чем европейские.

Рассмотрим две выборки, независимые по условию: $X$ - расход топлива европейских автомобилей, $Y$ - расход топлива американских автомобилей.

Тогда $H_{0}: \mu_{x} = 0.9 \mu_{Y}, \space H_{1}: \mu_{X} > 0.9 \mu_{Y} \Rightarrow$ рассмотрим случайную величину $\bar{X} - 0.9 \bar{Y} \overset{asy}{\sim} N(\mu_{Y} - 0.9 \mu_{Y}, \frac {\sigma_{X}^2} {n_{X}} + 0.81 \frac {\sigma_{Y}^2} {n_{Y}}) \overset{H_{0}}{\Rightarrow}$

$\overset{H_{0}}{\Rightarrow} z_{obs} = \frac {\bar{X} - \scriptstyle 0.9 \bar{Y}} {\sqrt{\frac {\hat{\sigma}_{X}^2} {n_{X}} + \scriptstyle 0.81 \frac {\hat{\sigma}_{Y}^2} {n_{Y}}}} \overset{asy}{\sim} N(0, 1), \space z_{crit} = z_{0.95}.$

In [18]:
X_cond = np.vectorize(lambda x: x in american_cars)(cars_df['Brand'])
X = cars_df[X_cond]['MPG_mean'].values

Y_cond = np.vectorize(lambda x: x in european_cars)(cars_df['Brand'])
Y = cars_df[Y_cond]['MPG_mean'].values

In [19]:
n_X = X.shape[0]
n_Y = Y.shape[0]

X_mean = X.mean()
Y_mean = Y.mean()

X_var = X.var(ddof=1)
Y_var = Y.var(ddof=1)

In [20]:
z_obs = (X_mean - 0.9 * Y_mean) / np.sqrt(X_var / n_X + 0.81 * Y_var / n_Y)
z_crit = sts.norm.ppf(0.95)
print(z_obs, z_crit)

# Видим, что наблюдаемое значение больше критического
z_obs > z_crit

4.735175066067651 1.6448536269514722


True

__Вывод:__ Гипотеза о том, что американские автомобили в среднем расходуют на 10% больше топлива, чем европейские, отвергается на данном уровне значимости.